### the purpose of the notebook is to build a scraper to harvest the Sony Xperia handsets product reviews. Then I will analyse the review in other notebook

#### load the neccessary modules

In [2]:
import json,mechanize,re
from lxml import html # using lxml to parse the html page instead of beautiful soup due to its speed
from fake_useragent import UserAgent # adding a randomly selected user agent and add it to the browser object

In [3]:
au = UserAgent(cache=False)
br =mechanize.Browser() 
br.set_handle_robots(False) # stop the robot to read the robot txt
br.addheaders = [('User-agent', 'rand_UserAgent')] # randomly add the user agent (Browsers eg: firefox) into the browser object

In [4]:
def get_web_data(weblink):
    webpage = br.open(weblink)
    webtext = webpage.read()
    webpage.close()
    parsed_page = html.fromstring(webtext)
    return parsed_page

### first go to the result page with the 'xperia' keyword search and grab all the product detail page url for within the first 10 pages and store the links in a text file

In [46]:
weblink_first_part = 'https://www.amazon.co.uk/gp/search/ref=sr_nr_n_1?fst=as%3Aoff&rh=n%3A356496011%2Ck%3Axperia&page='
weblink_second_part = '&keywords=xperia&ie=UTF8'
link_file = open('xperia_product.txt','a')
# weblink_first_part = 'https://www.amazon.co.uk/s/ref=sr_pg_2?rh=n%3A560798%2Cn%3A1340509031%2Cn%3A5362060031%2Cn%3A356496011%2Ck%3Asamsung&page='
# weblink_second_part = '&keywords=samsung&ie=UTF8'
# link_file = open('samsung_product.txt','a')

In [47]:
for i in range(1,11):
    parsed_page = get_web_data(weblink_first_part+str(i)+weblink_second_part)
#     parsed_page = html.fromstring(webtext)
    parsed_text = html.tostring(parsed_page)
    matched_result = re.findall('id="(result_\d.)',parsed_text)

    for result in matched_result:
        try:
            link_file.write(parsed_page.xpath('//*[@id="'+result.replace('"','')+'"]/div/div[2]/div/div/a/@href')[0])
            link_file.write('\n')
            
        except:
            pass
    print 'currently in page :'+str(i)
link_file.close()

currently in page :1
currently in page :2
currently in page :3
currently in page :4
currently in page :5
currently in page :6
currently in page :7
currently in page :8
currently in page :9
currently in page :10


### in each product detail page, search for the review page url and then go into the page. In the review page, identify the last page number, then from each review page scrape the review_author, review_date, review_star, review_title, review, plus product name and price information. Then store the data for further analysis.

In [ ]:
weblink_in = open('xperia_product.txt','r').read().split('\n')
review_data = open('xperia_product_reviews_testing.txt','a')

# weblink_in = open('samsung_product.txt','r').read().split('\n')
# review_data = open('samsung_product_reviews.txt','a')

product_name_review = '//*[@id="reviews-medley-footer"]/div[1]/a/@href'
main_web_domain = 'https://www.amazon.co.uk'
last_page_path = '//*[@id="cm_cr-pagination_bar"]/ul/li[7]/a/text()'
review_list_path = '//*[@id="cm_cr-review_list"]'

for link in weblink_in:
    try:
        parsed_page = get_web_data(link)
    
        product_name = parsed_page.xpath('//*[@id="productTitle"]/text()')[0].replace('\n','').strip()
        
        reviews_pagepart = parsed_page.xpath(product_name_review)[0]
        parsed_page = get_web_data(main_web_domain+reviews_pagepart)
        
        try:
            last_page = parsed_page.xpath(last_page_path)[0]
        except:
            last_page = '0'
            
        price = parsed_page.xpath('//*[@id="cm_cr-product_info"]/div/div[2]/div/div/div[2]/div[4]/span/span[3]/text()')[0].encode('utf-8')
        
        print 'passed stage 1'
        print 'last page : '+last_page
        
        try:
            for i in range(1,int(last_page)+1):
                try:
                    print i
                    if i == 1:
                        parsed_page = parsed_page
                    else:
                        parsed_page = get_web_data(main_web_domain+reviews_pagepart+'&pageNumber='+str(i))

                    print 'passed stage 2'

                    html_text = html.tostring(parsed_page.xpath(review_list_path)[0])
                #     sub_parsed_page = html.fromstring(html_text)
                    review_id_list = re.findall('<div id="customer_review-(.*)" class="a-section celwidget',html_text)

                    for review in review_id_list:

                        html_text = html.tostring(parsed_page.xpath('//*[@id="'+review+'"]')[0])
                        sub_parsed_page = html.fromstring(html_text)
                        review = sub_parsed_page.xpath('//*[@data-hook="review-body"]/text()')[0].replace('\n','').encode('utf-8')
                        review_title = sub_parsed_page.xpath('//*[@data-hook="review-title"]/text()')[0].encode('utf-8')
                        review_date =sub_parsed_page.xpath('//*[@data-hook="review-date"]/text()')[0]
                        review_author = sub_parsed_page.xpath('//*[@data-hook="review-author"]/text()')[0].encode('utf-8')
                        review_star = sub_parsed_page.xpath('//*[@data-hook="review-star-rating"]/span/text()')[0].replace(' out of 5 stars','')

        #                 print review_author+'|'+review_date+'|'+review_star+'|'+review_title+'|'+review
        #                 print "***"*10
                        review_data.write(product_name+'|'+price+'|'+review_author+'|'+review_date+'|'+review_star+'|'+review_title+'|'+review)
                        review_data.write('\n')
                except:
                    print "failed at link : "+main_web_domain+reviews_pagepart+str(i)
                    pass
        except:
            print "failed at : "+main_web_domain+reviews_pagepart
            pass
    except:
        print "failed at link: "+link
        
review_data.close()

passed stage 1
last page : 8
1
passed stage 2
2
passed stage 2
3
passed stage 2
4
passed stage 2
5
passed stage 2
6
passed stage 2
7
passed stage 2
8
passed stage 2
passed stage 1
last page : 11
1
passed stage 2
2
passed stage 2
3
passed stage 2
4
passed stage 2
5
failed at link : https://www.amazon.co.uk/Sony-Xperia-SIM-Free-Smart-Phone-Mineral-Black/product-reviews/B01L96EDEW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews5
6
passed stage 2
7
passed stage 2
8
passed stage 2
9
passed stage 2
10
passed stage 2
11
passed stage 2
passed stage 1
last page : 0
passed stage 1
last page : 9
1
passed stage 2
2
passed stage 2
3
passed stage 2
4
passed stage 2
5
failed at link : https://www.amazon.co.uk/Sony-Xperia-Compact-Smart-Phone-Universe-Black/product-reviews/B01L96ZBZ2/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews5
6
passed stage 2
7
passed stage 2
8
passed stage 2
9
passed stage 2
passed stage 1
last page : 0
